In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
import time
from tqdm import tqdm
from selenium.webdriver.common.by import By
from selenium import webdriver
from  selenium.webdriver.common.keys import Keys
from datetime import date
import numpy as np
from scripts.season_updater import *
from pycaret.classification import *

## Atualiza temporada

In [ ]:
atualiza_temporada('https://www.flashscore.com.br/basquete/eua/wnba/resultados/', 'data/wnba/wnba-2023.csv', 10)

## Scraping dos jogos do dia

In [ ]:
# Instanciando o Objeto ChromeOptions
options = webdriver.EdgeOptions()

# Passando algumas opções para esse ChromeOptions
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Criação do WebDriver do Chrome
wd_Chrome = webdriver.Edge(options=options)

try:
    # Com o WebDrive a gente consegue a pedir a página (URL)
    wd_Chrome.get("https://www.flashscore.com.br/basquete/eua/wnba/#/GrGdJSSg/table/overall")
    time.sleep(2)
except:
    print('Não tem jogos hoje')
    pass

# Para jogos do dia
jogos = wd_Chrome.find_element(By.CSS_SELECTOR, 'div.leagues--live')

# Pegando o ID dos Jogos
id_jogos = []
jogos = jogos.find_elements(By.CSS_SELECTOR,'div.event__match--scheduled')

for i in jogos:
    id_jogos.append(i.get_attribute("id"))

# Exemplo de ID de um jogo: 'g_1_Gb7buXVt'    
id_jogos = [i[4:] for i in id_jogos]

base_jogos = pd.DataFrame({})

for link in tqdm(id_jogos, total=len(id_jogos)):
    wd_Chrome.get(f'https://www.flashscore.com.br/jogo/{link}/#/resumo-de-jogo')
    
    Odds_H = 0
    Odds_A = 0
    Over_Line = 0
    Odds_Over = 0
    Odds_Under = 0
    HA_Line = 0
    HA_Odds_H = 0
    HA_Odds_A = 0

    # Pegando as Informacoes Básicas do Jogo
    try:
        Date = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__startTime').text.split(' ')[0]
        Season = Date.split('.')[2]
        Time = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__startTime').text.split(' ')[1]
        Home = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__home')
        Home = Home.find_element(By.CSS_SELECTOR,'div.participant__participantName').text
        Away = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__away')
        Away = Away.find_element(By.CSS_SELECTOR,'div.participant__participantName').text
        
        # Match Odds
        wd_Chrome.get(f'https://www.flashscore.com.br/jogo/{link}/#/comparacao-de-odds/home-away/tr-incluindo-prol')
        time.sleep(1)
        celulas = wd_Chrome.find_elements(By.CSS_SELECTOR,'div.ui-table__row')
        
        for celula in celulas:
            bookie = celula.find_element(By.CSS_SELECTOR,'img.prematchLogo')
            bookie = bookie.get_attribute('title')
            if (((bookie == 'bet365') and (Odds_H == 0)) or ((bookie == '1xBet') and (Odds_H == 0))):
                Odds_H = float(celula.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[0].text)
                Odds_A = float(celula.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[1].text)
            else:
                pass


        # Over/Under
        wd_Chrome.get(f'https://www.flashscore.com.br/jogo/{link}/#/comparacao-de-odds/acima-abaixo/tr-incluindo-prol')
        time.sleep(1)
        celulas = wd_Chrome.find_elements(By.CLASS_NAME,'ui-table__row')


        for celula in celulas:
            bookie = celula.find_element(By.CSS_SELECTOR, 'img.prematchLogo').get_attribute('title')
            Odds_Over = float(celula.find_elements(By.TAG_NAME,'span')[1].text)
            if (((bookie == 'bet365') and (Odds_Over >= 1.80)) or ((bookie == '1xBet') and (Odds_Over >= 1.80))):
                Over_Line = float(celula.find_elements(By.TAG_NAME,'span')[0].text)
                Odds_Under = float(celula.find_elements(By.TAG_NAME,'span')[2].text)
                break
            else:
                pass


        # Handicap
        wd_Chrome.get(f'https://www.flashscore.com.br/jogo/{link}/#/comparacao-de-odds/handicap-asiatico/tr-incluindo-prol')
        time.sleep(1)
        celulas = wd_Chrome.find_elements(By.CLASS_NAME,'ui-table__row')


        for celula in celulas:

            bookie = celula.find_element(By.CSS_SELECTOR, 'img.prematchLogo').get_attribute('title')
            HA_Odds_H = float(celula.find_elements(By.TAG_NAME,'span')[1].text)
            if (((bookie == 'bet365') and (HA_Odds_H <= 2.1)) or ((bookie == '1xBet') and (HA_Odds_H <= 2.1))):
                HA_Line = float(celula.find_elements(By.TAG_NAME,'span')[0].text)
                HA_Odds_A = float(celula.find_elements(By.TAG_NAME,'span')[2].text)
                break
            else:
                pass

    except:
        pass

    base_jogos.loc[base_jogos.shape[0],['Date','Season','Time','Home','Away','Odds_H','Odds_A','Over_Line','Odds_Over','Odds_Under','HA_Line','HA_Odds_H','HA_Odds_A']] = [
        Date, Season, Time, Home, Away, Odds_H, Odds_A, Over_Line, Odds_Over, Odds_Under, HA_Line, HA_Odds_H, HA_Odds_A]

wd_Chrome.quit()
base_jogos.drop(base_jogos[(base_jogos == 0).any(axis=1)].index, inplace=True)

In [ ]:
# base_jogos.to_csv("~/Desktop/MLB-jogos_do_dia.csv", index=False)
base_jogos

## Apostas do dia

#### Funções

In [ ]:
def calcular_mediacg(_df):
    final_df = _df.copy()

    def calcular_media(row):
        date, home, away = row['Date'], row['Home'], row['Away']

        df_cum = _df[(_df['Date'] < date) & ((_df['Home'] == home) | (_df['Away'] == home))].tail(7)
        if len(df_cum) == 7:
            df_cum.loc[(df_cum['Home'] == home), 'real_cg'] = df_cum['CustoGolHome']
            df_cum.loc[(df_cum['Away'] == home), 'real_cg'] = df_cum['CustoGolAway']
            media_CG = df_cum['real_cg'].mean()
            dp_CG = df_cum['real_cg'].std()
            cv_CG = dp_CG / media_CG
            final_df.loc[row.name, 'Avg_CG_H'] = media_CG
            final_df.loc[row.name, 'DP_CG_H'] = dp_CG
            final_df.loc[row.name, 'CV_CG_H'] = cv_CG
        else:
            final_df.loc[row.name, 'Avg_CG_H'] = 0
            final_df.loc[row.name, 'DP_CG_H'] = 0
            final_df.loc[row.name, 'CV_CG_H'] = 0

        df_cum = _df[(_df['Date'] < date) & ((_df['Home'] == away) | (_df['Away'] == away))].tail(7)
        if len(df_cum) == 7:
            df_cum.loc[(df_cum['Home'] == away), 'real_cg'] = df_cum['CustoGolHome']
            df_cum.loc[(df_cum['Away'] == away), 'real_cg'] = df_cum['CustoGolAway']
            media_CG = df_cum['real_cg'].mean()
            dp_CG = df_cum['real_cg'].std()
            cv_CG = dp_CG / media_CG
            final_df.loc[row.name, 'Avg_CG_A'] = media_CG
            final_df.loc[row.name, 'DP_CG_A'] = dp_CG
            final_df.loc[row.name, 'CV_CG_A'] = cv_CG
        else:
            final_df.loc[row.name, 'Avg_CG_A'] = 0
            final_df.loc[row.name, 'DP_CG_A'] = 0
            final_df.loc[row.name, 'CV_CG_A'] = 0

    _df.apply(calcular_media, axis=1)

    return final_df


################################################################################
# Prepara o _DF
################################################################################

def prepara_df(_df):
  _df = _df[(_df.Over_Line >= 5)]
  _df = _df[(_df['Home_Pts'] + _df['Away_Pts']) != _df['Over_Line']]
  _df = _df[_df['HA_Odds_A'] != 0]
  _df = _df[_df['Odds_H'] != 0]
  _df = _df[_df['Odds_A'] != 0]
  _df = _df[_df['Odds_Over'] != 0]
  _df = _df[_df['Odds_Under'] != 0]

  _df['P(H)'] = 1 / _df['Odds_H']
  _df['P(A)'] = 1 / _df['Odds_A']
  _df['P(O)'] = 1 / _df['Odds_Over']
  _df['P(U)'] = 1 / _df['Odds_Under']
  
  _df['P_Diff'] = ((1 / _df['Odds_H']) + (1 / _df['Odds_A'])) - 1
  _df['Porc_Over_Home'] = _df.groupby('Home')['Back_Over'].rolling(7).mean().reset_index(level=0, drop=True)
  _df['Porc_Over_Away'] = _df.groupby('Away')['Back_Over'].rolling(7).mean().reset_index(level=0, drop=True)
  _df['Porc_Over_Home']  = _df.groupby('Home')['Porc_Over_Home'].shift(1)
  _df['Porc_Over_Away']  = _df.groupby('Away')['Porc_Over_Away'].shift(1)
  _df['Porc_Over_Home'] = _df['Porc_Over_Home'].replace(np.nan, 0)
  _df['Porc_Over_Away'] = _df['Porc_Over_Away'].replace(np.nan, 0)
  _df.reset_index(inplace=True, drop=True)

  # Custo do gol
  _df['CustoGolHome'] = _df['Home_Pts'] / (1 / _df['Odds_H'])
  _df['CustoGolAway'] = _df['Away_Pts'] / (1 / _df['Odds_A'])
  _df['CustoGolHome'] = _df['CustoGolHome'].replace(np.inf, 0)
  _df['CustoGolAway'] = _df['CustoGolAway'].replace(np.inf, 0)
  _df.reset_index(drop=True, inplace=True)

  # Média, dp e cv do custo do gol
  _df = calcular_mediacg(_df)
  
  # Último custo do gol
  _df['Last_CG_H']  = _df.groupby('Home')['CustoGolHome'].shift(1)
  _df['Last_CG_A']  = _df.groupby('Away')['CustoGolAway'].shift(1)
  _df['Last_CG_H'] = _df['Last_CG_H'].replace(np.nan, 0)
  _df['Last_CG_A'] = _df['Last_CG_A'].replace(np.nan, 0)
  
  limit_up_h = _df.CustoGolHome.mean() + _df.CustoGolHome.std()
  limit_up_a = _df.CustoGolAway.mean() + _df.CustoGolAway.std()
  _df.loc[(_df['CustoGolHome'] > limit_up_h), 'Acima_Last_CG_H'] = 1
  _df.loc[(_df['CustoGolHome'] <= limit_up_h), 'Acima_Last_CG_H'] = 0
  _df.loc[(_df['CustoGolAway'] > limit_up_a), 'Acima_Last_CG_A'] = 1
  _df.loc[(_df['CustoGolAway'] <= limit_up_a), 'Acima_Last_CG_A'] = 0
  _df['Acima_Last_CG_H']  = _df.groupby('Home')['Acima_Last_CG_H'].shift(1)
  _df['Acima_Last_CG_A']  = _df.groupby('Away')['Acima_Last_CG_A'].shift(1)
  _df['Acima_Last_CG_H'] = _df['Acima_Last_CG_H'].replace(np.nan, 0)
  _df['Acima_Last_CG_A'] = _df['Acima_Last_CG_A'].replace(np.nan, 0)

  limit_down_h = _df.CustoGolHome.mean() - _df.CustoGolHome.std()
  limit_down_a = _df.CustoGolAway.mean() - _df.CustoGolAway.std()
  _df.loc[(_df['CustoGolHome'] < limit_down_h), 'Abaixo_Last_CG_H'] = 1
  _df.loc[(_df['CustoGolHome'] >= limit_down_h), 'Abaixo_Last_CG_H'] = 0
  _df.loc[(_df['CustoGolAway'] < limit_down_a), 'Abaixo_Last_CG_A'] = 1
  _df.loc[(_df['CustoGolAway'] >= limit_down_a), 'Abaixo_Last_CG_A'] = 0
  _df['Abaixo_Last_CG_H']  = _df.groupby('Home')['Abaixo_Last_CG_H'].shift(1)
  _df['Abaixo_Last_CG_A']  = _df.groupby('Away')['Abaixo_Last_CG_A'].shift(1)
  _df['Abaixo_Last_CG_H'] = _df['Abaixo_Last_CG_H'].replace(np.nan, 0)
  _df['Abaixo_Last_CG_A'] = _df['Abaixo_Last_CG_A'].replace(np.nan, 0)

  _df['CV_ML'] = (_df[['Odds_H', 'Odds_A']].std(axis=1)) / (_df[['Odds_H', 'Odds_A']].mean(axis=1))
  _df['CV_Over'] = (_df[['Odds_Over', 'Odds_Under']].std(axis=1)) / (_df[['Odds_Over', 'Odds_Under']].mean(axis=1))

  _df.pop('CustoGolHome')
  _df.pop('CustoGolAway')

  # Organiza as colunas do dataset
  _df = _df[['Date', 'Season', 'Season_Time', 'Time', 'Home', 'Away', 'Home_Pts',
       'Away_Pts', 'Odds_H', 'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under',
       'HA_Line', 'HA_Odds_H', 'HA_Odds_A', 'P(H)', 'P(A)', 'P(O)', 'P(U)', 'P_Diff',
       'Porc_Over_Home', 'Porc_Over_Away', 'Last_CG_H', 'Last_CG_A', 'Avg_CG_H', 'Avg_CG_A', 
       'DP_CG_H', 'DP_CG_A', 'CV_CG_H', 'CV_CG_A',
       'Acima_Last_CG_H', 'Acima_Last_CG_A', 'Abaixo_Last_CG_H', 'Abaixo_Last_CG_A', 'CV_ML',
       'CV_Over', 'Back_Home', 'PL_Home',
       'Back_Away', 'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H',
       'PL_HA_H', 'PL_HA_A']]
  
  return _df

In [ ]:
def atualiza_dia(jogos_hoje, temporada):
    jogos_hoje['P(H)'] = 1 / jogos_hoje['Odds_H']
    jogos_hoje['P(A)'] = 1 / jogos_hoje['Odds_A']
    jogos_hoje['P(O)'] = 1 / jogos_hoje['Odds_Over']
    jogos_hoje['P(U)'] = 1 / jogos_hoje['Odds_Under']
    jogos_hoje['CV_ML'] = (jogos_hoje[['Odds_H', 'Odds_A']].std(axis=1)) / (jogos_hoje[['Odds_H', 'Odds_A']].mean(axis=1))
    jogos_hoje['CV_Over'] = (jogos_hoje[['Odds_Over', 'Odds_Under']].std(axis=1)) / (jogos_hoje[['Odds_Over', 'Odds_Under']].mean(axis=1))
    jogos_hoje['P_Diff'] = ((1 / jogos_hoje['Odds_H']) + (1 / jogos_hoje['Odds_A'])) - 1

    temporada = prepara_df(temporada)

    df_home = temporada.groupby('Home').last()[['Porc_Over_Home', 'MediaCustoGolHome', 'Acima_Last_CG_H', 'Abaixo_Last_CG_H', 'CustoGolHome', 'Last_CG_H']]
    df_away = temporada.groupby('Away').last()[['Porc_Over_Away', 'MediaCustoGolAway', 'Acima_Last_CG_A', 'Abaixo_Last_CG_A', 'CustoGolAway', 'Last_CG_A']]

    jogos_hoje = pd.merge(jogos_hoje, df_home, on='Home', how='left')
    jogos_hoje = pd.merge(jogos_hoje, df_away, on='Away', how='left')

    jogos_hoje.rename(columns={'CustoGolHome':'Last_CG_Home', 'CustoGolAway':'Last_CG_Away'}, inplace=True)

    jogos_hoje = jogos_hoje.reset_index(drop=True)

    return jogos_hoje

In [ ]:
df = base_jogos.copy()
temporada = pd.read_csv('data/wnba/wnba-2023.csv')
df = atualiza_dia(df, temporada)

#### Jogos Método Back Home de hoje

In [ ]:
# Duplicando o dataset para Back Home
df_back_home = df[(df['Odds_H'] >= 1.6)]

model_back_home = load_model('ML/back_home')
previsoes = predict_model(model_back_home, df_back_home)
df_back_home = previsoes[previsoes['prediction_label'] == 1]

df_back_home['Bet'] = 'Back Home'
df_back_home = df_back_home[['Time', 'Home', 'Away', 'Odds_H', 'Bet']]
df_back_home.sort_values('Time', inplace=True)
df_back_home.reset_index(drop=True, inplace=True)

hoje = date.today()
# df_back_home.to_excel(f'~/Desktop/MLB_over_{hoje}.xlsx', index=False)

print(f'{len(df_back_home)} jogos para o método Back Home de hoje ({hoje})')
df_back_home